# Power your products with ChatGPT and your own data

This is a walkthrough taking readers through how to build starter Q&A and Chatbot applications using the ChatGPT API and their own data. 

It is laid out in these sections:
- **Setup:** 
    - Initiate variables and source the data
- **Lay the foundations:**
    - Set up the vector database to accept vectors and data
    - Load the dataset, chunk the data up for embedding and store in the vector database
- **Make it a product:**
    - Add a retrieval step where users provide queries and we return the most relevant entries
    - Summarise search results with GPT-3
    - Test out this basic Q&A app in Streamlit
- **Build your moat:**
    - Create an Assistant class to manage context and interact with our bot
    - Use the Chatbot to answer questions using semantic search context
    - Test out this basic Chatbot app in Streamlit
    
Upon completion, you have the building blocks to create your own production chatbot or Q&A application using OpenAI APIs and a vector database.

This notebook was originally presented with [these slides](https://drive.google.com/file/d/1dB-RQhZC_Q1iAsHkNNdkqtxxXqYODFYy/view?usp=share_link), which provide visual context for this journey.

In [1]:
%load_ext autoreload
%autoreload 2

## Setup

First we'll setup our libraries and environment variables

In [2]:
import openai
import os
import requests
import numpy as np
import pandas as pd
from typing import Iterator
import tiktoken
import textract
from numpy import array, average

from database import get_redis_connection

# Set our default models and chunking size
from config import COMPLETIONS_MODEL, EMBEDDINGS_MODEL, CHAT_MODEL, TEXT_EMBEDDING_CHUNK_SIZE, VECTOR_FIELD_NAME,OPENAI_KEY

# Ignore unclosed SSL socket warnings - optional in case you get these errors
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ImportWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

openai.api_key = OPENAI_KEY

In [3]:
pd.set_option('display.max_colwidth', 0)

In [4]:
data_dir = os.path.join(os.curdir,'data')
pdf_files = sorted([x for x in os.listdir(data_dir) if 'DS_Store' not in x])
pdf_files

["FIA Practice Directions - Competitor's Staff Registration System.pdf",
 'f1040.pdf',
 'fia_2022_formula_1_sporting_regulations_-_issue_9_-_2022-10-19_0.pdf',
 'fia_2023_formula_1_technical_regulations_-_issue_4_-_2022-12-07.pdf',
 'fia_f1_power_unit_financial_regulations_issue_1_-_2022-08-16.pdf',
 'fia_formula_1_financial_regulations_iss.13.pdf',
 'i1040gi.pdf',
 'i1040tt.pdf',
 'iw9.pdf']

## Laying the foundations

### Storage

We're going to use Redis as our database for both document contents and the vector embeddings. You will need the full Redis Stack to enable use of Redisearch, which is the module that allows semantic search - more detail is in the [docs for Redis Stack](https://redis.io/docs/stack/get-started/install/docker/).

To set this up locally, you will need to install Docker and then run the following command: ```docker run -d --name redis-stack -p 6379:6379 -p 8001:8001 redis/redis-stack:latest```.

The code used here draws heavily on [this repo](https://github.com/RedisAI/vecsim-demo).

After setting up the Docker instance of Redis Stack, you can follow the below instructions to initiate a Redis connection and create a Hierarchical Navigable Small World (HNSW) index for semantic search.

In [5]:
# Setup Redis
from redis import Redis
from redis.commands.search.query import Query
from redis.commands.search.field import (
    TextField,
    VectorField,
    NumericField
)
from redis.commands.search.indexDefinition import (
    IndexDefinition,
    IndexType
)

redis_client = get_redis_connection()

In [6]:
# Constants
VECTOR_DIM = 1536 #len(data['title_vector'][0]) # length of the vectors
#VECTOR_NUMBER = len(data)                 # initial number of vectors
PREFIX = "sportsdoc"                            # prefix for the document keys
DISTANCE_METRIC = "COSINE"                # distance metric for the vectors (ex. COSINE, IP, L2)

In [7]:
# Create search index

# Index
INDEX_NAME = "f1-index"           # name of the search index
VECTOR_FIELD_NAME = 'content_vector'

# Define RediSearch fields for each of the columns in the dataset
# This is where you should add any additional metadata you want to capture
filename = TextField("filename")
text_chunk = TextField("text_chunk")
file_chunk_index = NumericField("file_chunk_index")

# define RediSearch vector fields to use HNSW index

text_embedding = VectorField(VECTOR_FIELD_NAME,
    "HNSW", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC
    }
)
# Add all our field objects to a list to be created as an index
fields = [filename,text_chunk,file_chunk_index,text_embedding]

In [8]:
redis_client.ping()

True

In [9]:
# Optional step to drop the index if it already exists
#redis_client.ft(INDEX_NAME).dropindex()

# Check if index exists
try:
    redis_client.ft(INDEX_NAME).info()
    print("Index already exists")
except Exception as e:
    print(e)
    # Create RediSearch Index
    print('Not there yet. Creating')
    redis_client.ft(INDEX_NAME).create_index(
        fields = fields,
        definition = IndexDefinition(prefix=[PREFIX], index_type=IndexType.HASH)
    )

Index already exists


### Ingestion

We'll load up our PDFs and do the following
- Initiate our tokenizer
- Run a processing pipeline to:
    - Mine the text from each PDF
    - Split them into chunks and embed them
    - Store them in Redis

In [10]:
# The transformers.py file contains all of the transforming functions, including ones to chunk, embed and load data
# For more details the file and work through each function individually
from transformers import handle_file_string

In [11]:
%%time
# This step takes about 5 minutes

# Initialise tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

# Process each PDF file and prepare for embedding
for pdf_file in pdf_files:
    
    pdf_path = os.path.join(data_dir,pdf_file)
    print(pdf_path)
    
    # Extract the raw text from each PDF using textract
    text = textract.process(pdf_path, method='pdfminer')
    
    # Chunk each document, embed the contents and load to Redis
    handle_file_string((pdf_file,text.decode("utf-8")),tokenizer,redis_client,VECTOR_FIELD_NAME,INDEX_NAME)

./data/FIA Practice Directions - Competitor's Staff Registration System.pdf
./data/f1040.pdf
./data/fia_2022_formula_1_sporting_regulations_-_issue_9_-_2022-10-19_0.pdf
./data/fia_2023_formula_1_technical_regulations_-_issue_4_-_2022-12-07.pdf
./data/fia_f1_power_unit_financial_regulations_issue_1_-_2022-08-16.pdf
./data/fia_formula_1_financial_regulations_iss.13.pdf
./data/i1040gi.pdf
./data/i1040tt.pdf
./data/iw9.pdf
CPU times: user 13.9 s, sys: 2.85 s, total: 16.7 s
Wall time: 3min 4s


In [12]:
# Check that our docs have been inserted
redis_client.ft(INDEX_NAME).info()['num_docs']

'1713'

## Make it a product

Now we can test that our search works as intended by:
- Querying our data in Redis using semantic search and verifying results
- Adding a step to pass the results to GPT-3 for summarisation

In [13]:
from database import get_redis_results

In [15]:
%%time

f1_query='what are some deductions for 1099 form?'

result_df = get_redis_results(redis_client,f1_query,index_name=INDEX_NAME)
result_df.head(2)

CPU times: user 7.37 ms, sys: 2.41 ms, total: 9.78 ms
Wall time: 454 ms


,id,result,certainty
0,0,"to figure For more details, see Pub. 535. If you qualify to take the deduction, use the Self-Employed Health Insurance Deduction Worksheet the amount you can deduct. Exceptions. Use Pub. 535 instead of the Self-Employed Health Insurance De- duction Worksheet in these instructions to figure your deduction if any of the following applies. • You had more than one source of income subject to self-employment tax. • You file Form 2555. • You are using amounts paid for qualified long-term care insurance to figure the deduction. Use Pub. 974 instead of the work- sheet in these instructions if the insur- ance plan was considered to be estab- lished under your business and was ob- tained through the Marketplace, and ad- vance payments of the premium tax credit were made or you are claiming the premium tax credit. Line 18 Penalty on Early Withdrawal of Savings The Form 1099-INT or Form 1099-OID you received will show the amount of any penalty you were charged. Lines 19a, 19b, and 19c Alimony Paid Line 19a If you made payments to or for your spouse or former spouse under a divorce or separation agreement entered into on or before December 31, 2018, you may be able to take this deduction. You can't take a deduction for alimony payments you made to or for your spouse if you entered into your divorce or separation agreement after December 31, 2018, or if you entered into the agreement on or before December 31, 2018, and the agreement was changed after December 31, 2018, to expressly provide that ali- mony received is not included in your former spouse's income. Use Tax Topic 452 or see Pub. 504.",0.146604061127
1,1,"Attach Form(s) W-2G to Form 1040 or 1040-SR if any federal income tax was withheld. Line 8c Cancellation of debt. Enter on line 8c any canceled debt. Canceled debt may be shown in box 2 of Form 1099-C. However, part or all of your income from cancellation of debt may be non- taxable. See Pub. 4681 or go to IRS.gov and enter “canceled debt” or “foreclo- sure” in the search box. Line 8d Foreign earned income exclusion and housing exclusion from Form 2555. Enter the amount of your foreign earned income and housing exclusion from Form 2555, line 45. Enter the amount in the preprinted parentheses (as a negative -85- number). The amount from Form 2555, line 45, will be subtracted from the other amounts of income listed on lines 8a through 8c and lines 8e through 8z. Complete the Foreign Earned Income Tax Worksheet if you enter an amount on Form 2555, line 45. Line 8e Income from Form 8853. Enter on line 8e the total of the amounts from Form 8853, lines 8, 12, and 26. See Pub. 969. You may have to pay an addi- tional tax if you received a tax- able distribution from an Arch- er MSA or Medicare Advantage MSA. See the Instructions for Form 8853. Line 8f Income from Form 8889. Enter on line 8f the total of the amounts from Form 8889, lines 16 and 20. You may have to pay an addi- tional tax if you received a tax- able distribution from a health savings account. See the Instructions for Form 8889. Line 8h Jury duty pay.",0.154962956905


In [16]:
# Build a prompt to provide the original query, the result and ask to summarise for the user
summary_prompt = '''Summarise this result in a bulleted list to answer the search query a customer has sent.
Search query: SEARCH_QUERY_HERE
Search result: SEARCH_RESULT_HERE
Summary:
'''
summary_prepped = summary_prompt.replace('SEARCH_QUERY_HERE',f1_query).replace('SEARCH_RESULT_HERE',result_df['result'][0])
summary = openai.Completion.create(engine=COMPLETIONS_MODEL,prompt=summary_prepped,max_tokens=500)
# Response provided by GPT-3
print(summary['choices'][0]['text'])

- Self-employment deductions available for health insurance expenses, as outlined in Pub. 535
- Exceptions to deducting health insurance expenses may apply, as outlined in Pub. 535
- Deduction for penalty on early withdrawal of savings, as shown on Form 1099-INT or 1099-OID
- Deduction for alimony payments made before and/or after December 31, 2018, as outlined in Tax Topic 452 or Pub. 504


### Search

Now that we've got our knowledge embedded and stored in Redis, we can now create an internal search application. Its not sophisticated but it'll get the job done for us.

In the directory containing this app, execute ```streamlit run search.py```. This will open up a Streamlit app in your browser where you can ask questions of your embedded data.

__Example Questions__:
- what is the cost cap for a power unit in 2023
- what should competitors include on their application form

## Build your moat

The Q&A was useful, but fairly limited in the complexity of interaction we can have - if the user asks a sub-optimal question, there is no assistance from the system to prompt them for more info or conversation to lead them down the right path.

For the next step we'll make a Chatbot using the Chat Completions endpoint, which will:
- Be given instructions on how it should act and what the goals of its users are
- Be supplied some required information that it needs to collect
- Go back and forth with the customer until it has populated that information
- Say a trigger word that will kick off semantic search and summarisation of the response

For more details on our Chat Completions endpoint and how to interact with it, please check out the docs [here](https://platform.openai.com/docs/guides/chat).

### Framework

This section outlines a basic framework for working with the API and storing context of previous conversation "turns". Once this is established, we'll extend it to use our retrieval endpoint.

In [16]:
# A basic example of how to interact with our ChatCompletion endpoint
# It requires a list of "messages", consisting of a "role" (one of system, user or assistant) and "content"
question = 'How can you help me'


completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": question}
  ]
)
print(f"{completion['choices'][0]['message']['role']}: {completion['choices'][0]['message']['content']}")

assistant: As an AI language model, I can help you in various ways such as:

1. Answering your questions: You can ask me questions about different topics, and I will try my best to provide you with accurate and relevant answers.

2. Proofreading and editing: I can help you proofread and edit your texts, ensuring that your writing is free from grammatical errors and spelling mistakes.

3. Translation: If you need assistance with translation, I can translate texts from one language to another.

4. Brainstorming ideas: If you need ideas for a project, I can help you brainstorm ideas, providing you with creative suggestions.

5. Providing information: I can help you find information on different topics or assist you in research.

6. Personal assistance: As a language model, I can assist you in daily tasks such as setting reminders, checking your calendar, and sending emails.


In [17]:
from termcolor import colored

# A basic class to create a message as a dict for chat
class Message:
    
    
    def __init__(self,role,content):
        
        self.role = role
        self.content = content
        
    def message(self):
        
        return {"role": self.role,"content": self.content}
        
# Our assistant class we'll use to converse with the bot
class Assistant:
    
    def __init__(self):
        self.conversation_history = []

    def _get_assistant_response(self, prompt):
        
        try:
            completion = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=prompt
            )
            
            response_message = Message(completion['choices'][0]['message']['role'],completion['choices'][0]['message']['content'])
            return response_message.message()
            
        except Exception as e:
            
            return f'Request failed with exception {e}'

    def ask_assistant(self, next_user_prompt, colorize_assistant_replies=True):
        [self.conversation_history.append(x) for x in next_user_prompt]
        assistant_response = self._get_assistant_response(self.conversation_history)
        self.conversation_history.append(assistant_response)
        return assistant_response
            
        
    def pretty_print_conversation_history(self, colorize_assistant_replies=True):
        for entry in self.conversation_history:
            if entry['role'] == 'system':
                pass
            else:
                prefix = entry['role']
                content = entry['content']
                output = colored(prefix +':\n' + content, 'green') if colorize_assistant_replies and entry['role'] == 'assistant' else prefix +':\n' + content
                print(output)

In [18]:
# Initiate our Assistant class
conversation = Assistant()

# Create a list to hold our messages and insert both a system message to guide behaviour and our first user question
messages = []
system_message = Message('system','You are a helpful Tax assistant who knows the tax rules')
user_message = Message('user','What can you do to help me')
messages.append(system_message.message())
messages.append(user_message.message())
messages

[{'role': 'system',
  'content': 'You are a helpful Tax assistant who knows the tax rules'},
 {'role': 'user', 'content': 'What can you do to help me'}]

In [19]:
# Get back a response from the Chatbot to our question
response_message = conversation.ask_assistant(messages)
print(response_message['content'])

As a tax assistant, I can provide you with tax advice and guidance on how to navigate the tax system. This includes helping you understand:

1. Your tax obligations: I can help you understand your tax obligations based on your income, deductions, and credits.

2. Tax deductions and credits: I can help you identify tax deductions and credits that you may be eligible for to help you minimize your tax liability.

3. Tax laws and regulations: I can help you stay up-to-date with the latest tax laws and regulations that may impact your tax situation.

4. Tax planning: I can help you with tax planning strategies to help you optimize your taxes and minimize your tax burden.

5. Tax submissions and filings: I can help you with the preparation and submission of your tax returns to ensure that they are accurate and compliant with applicable tax laws and regulations.

6. Tax questions and concerns: I can address any tax-related questions or concerns that you may have and provide guidance on how to

In [20]:
next_question = 'Tell me more about option 3'

# Initiate a fresh messages list and insert our next question
messages = []
user_message = Message('user',next_question)
messages.append(user_message.message())
response_message = conversation.ask_assistant(messages)
print(response_message['content'])

Understanding tax laws and regulations is an essential aspect of navigating the tax system. Tax laws and regulations are dynamic and can change significantly over time. As a tax assistant, I can provide you with up-to-date information on current tax laws and how they pertain to your tax situation.

For example, there may be changes to tax laws in a given year that affect the deductions or credits you are eligible for. I can help you navigate these changes and identify any new opportunities to maximize your savings.

I can also help you understand complex tax regulations that may apply to your business, such as tax compliance requirements or rules surrounding foreign income tax reporting.

Overall, a thorough understanding of tax laws and regulations can help you make informed tax decisions and ensure that your tax filings are accurate and compliant. As a tax assistant, I can ensure you stay updated on the latest changes and answer any questions you may have about them.


In [21]:
# Print out a log of our conversation so far
conversation.pretty_print_conversation_history()

user:
What can you do to help me
assistant:
As a tax assistant, I can provide you with tax advice and guidance on how to navigate the tax system. This includes helping you understand:

1. Your tax obligations: I can help you understand your tax obligations based on your income, deductions, and credits.

2. Tax deductions and credits: I can help you identify tax deductions and credits that you may be eligible for to help you minimize your tax liability.

3. Tax laws and regulations: I can help you stay up-to-date with the latest tax laws and regulations that may impact your tax situation.

4. Tax planning: I can help you with tax planning strategies to help you optimize your taxes and minimize your tax burden.

5. Tax submissions and filings: I can help you with the preparation and submission of your tax returns to ensure that they are accurate and compliant with applicable tax laws and regulations.

6. Tax questions and concerns: I can address any tax-related questions or concerns that

### Knowledge retrieval

Now we'll extend the class to call a downstream service when a stop sequence is spoken by the Chatbot.

The main changes are:
- The system message is more comprehensive, giving criteria for the Chatbot to advance the conversation
- Adding an explicit stop sequence for it to use when it has the info it needs
- Extending the class with a function ```_get_search_results``` which sources Redis results

In [22]:
# Updated system prompt requiring Question and Year to be extracted from the user
system_prompt = '''
You are a helpful Tax knowledge base assistant. You need to capture a Question and Year from each customer.
The Question is their query on Taxes, and the Year is the year of the applicable Tax season.
If they haven't provided the Year, ask them for it again.
Once you have the Year, say "searching for answers".

Example 1:

User: I'd like to know the deductions I am eligible for with an income of 100000 dollars , married and 2 kids

Assistant: Certainly, what year would you like this for?

User: 2023 please.

Assistant: Searching for answers.
'''

# New Assistant class to add a vector database call to its responses
class RetrievalAssistant:
    
    def __init__(self):
        self.conversation_history = []  

    def _get_assistant_response(self, prompt):
        
        try:
            completion = openai.ChatCompletion.create(
              model=CHAT_MODEL,
              messages=prompt,
              temperature=0.1
            )
            
            response_message = Message(completion['choices'][0]['message']['role'],completion['choices'][0]['message']['content'])
            return response_message.message()
            
        except Exception as e:
            
            return f'Request failed with exception {e}'
    
    # The function to retrieve Redis search results
    def _get_search_results(self,prompt):
        latest_question = prompt
        search_content = get_redis_results(redis_client,latest_question,INDEX_NAME)['result'][0]
        return search_content
        

    def ask_assistant(self, next_user_prompt):
        [self.conversation_history.append(x) for x in next_user_prompt]
        assistant_response = self._get_assistant_response(self.conversation_history)
        
        # Answer normally unless the trigger sequence is used "searching_for_answers"
        if 'searching for answers' in assistant_response['content'].lower():
            question_extract = openai.Completion.create(model=COMPLETIONS_MODEL,prompt=f"Extract the user's latest question and the year for that question from this conversation: {self.conversation_history}. Extract it as a sentence stating the Question and Year")
            search_result = self._get_search_results(question_extract['choices'][0]['text'])
            
            # We insert an extra system prompt here to give fresh context to the Chatbot on how to use the Redis results
            # In this instance we add it to the conversation history, but in production it may be better to hide
            self.conversation_history.insert(-1,{"role": 'system',"content": f"Answer the user's question using this content: {search_result}. If you cannot answer the question, say 'Sorry, I don't know the answer to this one'"})
            #[self.conversation_history.append(x) for x in next_user_prompt]
            
            assistant_response = self._get_assistant_response(self.conversation_history)
            print(next_user_prompt)
            print(assistant_response)
            self.conversation_history.append(assistant_response)
            return assistant_response
        else:
            self.conversation_history.append(assistant_response)
            return assistant_response
            
        
    def pretty_print_conversation_history(self, colorize_assistant_replies=True):
        for entry in self.conversation_history:
            if entry['role'] == 'system':
                pass
            else:
                prefix = entry['role']
                content = entry['content']
                output = colored(prefix +':\n' + content, 'green') if colorize_assistant_replies and entry['role'] == 'assistant' else prefix +':\n' + content
                #prefix = entry['role']
                print(output)

In [25]:
conversation = RetrievalAssistant()
messages = []
system_message = Message('system',system_prompt)
user_message = Message('user','what deductions  am I eligible for with an income of 100000 dollars  and married')
messages.append(system_message.message())
messages.append(user_message.message())
response_message = conversation.ask_assistant(messages)
response_message

{'role': 'assistant',
 'content': 'Certainly, what year would you like this for?'}

In [26]:
messages = []
user_message = Message('user','For 2023 please.')
messages.append(user_message.message())
response_message = conversation.ask_assistant(messages)
#response_message

[{'role': 'user', 'content': 'For 2023 please.'}]
{'role': 'assistant', 'content': "I'm sorry, but the information I provided earlier was not related to your question. To answer your question, the deductions you are eligible for with an income of $100,000 and married will depend on several factors, such as your filing status, the deductions you choose to take, and the tax laws in effect for the tax year in question. Some common deductions that you may be eligible for include the standard deduction, mortgage interest, charitable contributions, and state and local taxes. However, to get a more accurate answer, I would need more information about your specific situation. Can you please provide more details about your question?"}


In [27]:
conversation.pretty_print_conversation_history()

user:
what deductions  am I eligible for with an income of 100000 dollars  and married
assistant:
Certainly, what year would you like this for?
user:
For 2023 please.
assistant:
I'm sorry, but the information I provided earlier was not related to your question. To answer your question, the deductions you are eligible for with an income of $100,000 and married will depend on several factors, such as your filing status, the deductions you choose to take, and the tax laws in effect for the tax year in question. Some common deductions that you may be eligible for include the standard deduction, mortgage interest, charitable contributions, and state and local taxes. However, to get a more accurate answer, I would need more information about your specific situation. Can you please provide more details about your question?


### Chatbot

Now we'll put all this into action with a real (basic) Chatbot.

In the directory containing this app, execute ```streamlit run chat.py```. This will open up a Streamlit app in your browser where you can ask questions of your embedded data. 

__Example Questions__:
- what is the cost cap for a power unit in 2023
- what should competitors include on their application form
- how can a competitor be disqualified

### Consolidation

Over the course of this notebook you have:
- Laid the foundations of your product by embedding our knowledge base
- Created a Q&A application to serve basic use cases
- Extended this to be an interactive Chatbot

These are the foundational building blocks of any Q&A or Chat application using our APIs - these are your starting point, and we look forward to seeing what you build with them!